In [1]:
try:
    from google.colab import drive
    IN_COLAB = True
except:
    IN_COLAB = False


if IN_COLAB:
    drive.mount("/content/drive/")
    %cd ./drive/MyDrive/Projekte/tsf/

IN_COLAB

False

In [2]:
import pandas as pd
import src
import importlib
importlib.reload(src)

from torch.utils.data import DataLoader

In [3]:
CFG = {
    "looking_back_window": 48,
    "forecast_horizon": 24,
    "batch_size": 512,
    "dev_size": 0.1,
    "num_workers": 2 if IN_COLAB else 0,
    "lr": 0.01,
    "dropout": 0,
    "additional_blocks": 0,
    "hidden_channels_factor": 1
}

### Data

In [4]:
dataset, data_loader = src.get_ett(
    looking_back_window=CFG["looking_back_window"],
    forecast_horizon=CFG["forecast_horizon"],
    num_workers=CFG["num_workers"],
    batch_size=CFG["batch_size"]
)
len(dataset), len(data_loader)

(2, 2)

In [5]:
net = src.TCN( 
    CFG["looking_back_window"],
    CFG["forecast_horizon"],
    dataset["train"].features, 
    dropout=CFG["dropout"],
    additional_blocks=CFG["additional_blocks"],
    hidden_channels_factor=CFG["hidden_channels_factor"]
)
print(net.amount_parameters)
print(net)

11092
TCN(
  (m): Sequential(
    (0): ResidualBlock(
      (res): Conv1d(1, 24, kernel_size=(1,), stride=(1,))
      (m): Sequential(
        (0): ConstantPad1d(padding=(2, 0), value=0)
        (1): Conv1d(1, 24, kernel_size=(3,), stride=(1,))
        (2): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): Dropout(p=0, inplace=False)
        (5): ConstantPad1d(padding=(2, 0), value=0)
        (6): Conv1d(24, 24, kernel_size=(3,), stride=(1,))
        (7): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (g_0): ReLU()
    (d_0): Dropout(p=0, inplace=False)
    (1): ResidualBlock(
      (m): Sequential(
        (0): ConstantPad1d(padding=(4, 0), value=0)
        (1): Conv1d(24, 24, kernel_size=(3,), stride=(1,), dilation=(2,))
        (2): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): Dropout(p=0, inplace=False)

In [ ]:
trainer = src.Trainer()
trainer.fit(net, data_loader, epochs=100)

(1) train 	 l1=0.31
(1) dev 	 l1=0.23
(2) train 	 l1=0.26


In [ ]:
y_true, y_pred = trainer.predict(net, data_loader)
len(y_true), len(y_pred)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, figsize=(20, 12))
zoom_idx, zoom_size = 500, 24*7


axs[0].plot(y_true, label="y_true")
axs[0].plot(y_pred, label="y_pred")
axs[0].legend()
axs[1].plot(y_true[zoom_idx:zoom_idx+zoom_size], label="y_true")
axs[1].plot(y_pred[zoom_idx:zoom_idx+zoom_size], label="y_pred")
axs[1].legend()